In [6]:
import pandas as pd

First read the data into dataframe. Encoding needs to be paid attention to.

In [7]:
df = pd.read_csv('Seattle_Hotels.csv', encoding='latin-1')
df.head()

,name,address,desc
0,Hilton Garden Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA","Located on the southern tip of Lake Union, the..."
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA","Located in the city's vibrant core, the Sherat..."
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101","Located in the heart of downtown Seattle, the ..."
3,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101",What?s near our hotel downtown Seattle locatio...
4,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA",Situated amid incredible shopping and iconic a...


In [9]:
from pyecharts.charts import Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType, CurrentConfig, NotebookType

CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

bar = (Bar(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
        .add_xaxis(['shirts', 'sweaters', 'pants', 'socks'])
        .add_yaxis("A", [5, 20, 35, 10])
        .add_yaxis("B", [15, 6, 45, 66])
        .set_global_opts(title_opts=opts.TitleOpts(title="MainSubject", subtitle="Subtitle")))
bar.load_javascript()
bar.render_notebook()